# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

## Construindo Um Algoritmo Para Rede Neural Multilayer Perceptron

### Forward Propagation

Para definir sua rede, você precisará definir a ordem das operações para seus nós. Dado que a entrada para algum nó depende das saídas dos outros, você precisa construir o grafo de tal forma que todas as dependências de entrada para cada nó sejam resolvidas antes de tentar executar o cálculo. Esta é uma técnica chamada de organização topológica (topological sort).

A função topological_sort() implementa a classificação topológica usando o Algoritmo de Kahn. Os detalhes deste método não são importantes, mas basicamente o método topological_sort() retorna uma lista ordenada de nós em que todos os cálculos podem ser executados em série. O método topological_sort() assume um feed_dict, que é como inicialmente configuramos um valor para um nó de entrada. O feed_dict é representado pela estrutura de dados do dicionário Python (você deve se lembrar que usamos feed_dict frequentemente com o TensorFlow).

### Definindo as classes dos nós e as funções de operações

In [1]:
class Neuronio(object):
    def __init__(self, nodes_entrada = []):
        self.nodes_entrada = nodes_entrada
        self.nodes_saida = []
        self.valor = None
        for n in self.nodes_entrada:
            n.nodes_saida.append(self)

    def forward(self):
        """
        Forward propagation.

        Calcule o valor de saída com base em `nodes_entrada` e
        armazene o resultado no self.valor.
        """
        raise NotImplemented


class Input(Neuronio):
    def __init__(self):
        Neuronio.__init__(self)

    def forward(self, valor = None):
        if valor is not None:
            self.valor = valor

            
class Add(Neuronio):
    def __init__(self, *inputs):
        Neuronio.__init__(self, inputs)

    def forward(self):
        x_value = self.nodes_entrada[0].valor
        y_value = self.nodes_entrada[1].valor
        self.valor = x_value + y_value

        
def topological_sort(feed_dict):
    """
    Classifica os nós em ordem topológica usando o Algoritmo de Kahn.

    `Feed_dict`: um dicionário em que a chave é um nó ` Input` e o valor é o respectivo feed de valor para esse nó.

    Retorna uma lista de nós ordenados.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.nodes_saida:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.valor = feed_dict[n]

        L.append(n)
        for m in n.nodes_saida:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_node, sorted_nodes):
    """
   Executa uma passagem para a frente através de uma lista de nós ordenados.

     Argumentos:

         `Output_node`: Um nó no grafo, deve ser o nó de saída.
         `Sorted_nodes`: uma lista topologicamente ordenada de nós.

     Retorna o valor do Nó de saída
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.valor

Por que usamos x_value e y_value para a leitura da matriz nodes_entrada? Vejamos: o objeto nodes_entrada é definido quando o nó é instanciado. Não usamos a classe Neuronio diretamente, em vez disso, usamos a classe Add, que é uma subclasse da classe Neuronio. O construtor da classe Add é responsável por passar os nós de entrada para a classe Neuronio, o que acontece na função init da classe Add. Mas por que usamos self.nodes_entrada[0].valor para manter o valor das entradas (usando a indexação [0], [1], etc...)? Fazemos isso por conta da ordenação topológica. Para cada nó da classe Input(), os nós são definidos diretamente quando você executa topological_sort:

if isinstance(n, Input):

    n.valor = feed_dict[n]
            
Ou seja, ao executar o grafo (na célula abaixo), definimos o feed_dict com os valores de entrada, passamos isso como parâmetro para a função topologica_sort() e ela retorna para a classe Input, a lista de valores. Por isso precisamos definir self.nodes_entrada[0].valor para ler cada um dos valores. Mas isso vale apenas para a classe Input. Para outras classes, o valor de n.valor é definido na passagem para frente:

for n in sorted_nodes:
    
    n.forward()

### Executando o Grafo

In [2]:
# Este script cria e executa o grafo 

# Define os inputs
x, y = Input(), Input()

# Chamada a função Add()
f = Add(x, y)

# Define o feed_dict
feed_dict = {x: 7, y: 8}

# Ordena as entradas para execução
sorted_neurons = topological_sort(feed_dict)

# Gera o output com o forward_pass
output = forward_pass(f, sorted_neurons)

# Print
print("{} + {} = {}".format(feed_dict[x], feed_dict[y], output))

7 + 8 = 15


## Fim